Import dependencies

In [5]:
from openai import OpenAI
from dotenv import load_dotenv
from links_extractor import fetch_website_links
import os

load_dotenv()

True

Define Open AI keys

In [6]:
OPENROUTER_API_KEY= os.getenv("OPENROUTER_API_KEY")
OPENROUTER_BASE_URL= os.getenv("OPENROUTER_BASE_URL")


Rail guards check

In [7]:
if OPENROUTER_API_KEY:
    print("API_KEY found")
else:
    print("No API_KEY found")
    exit(1)

API_KEY found


Define OpenAI client library

In [8]:
client = OpenAI(base_url=OPENROUTER_BASE_URL, api_key=OPENROUTER_API_KEY)

Define system and user prompt 

In [9]:

system_one_shot_prompt = """You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [14]:
def user_prompts(url):
    user_prompt = f"""
    Here is the list of links on the website {url} -
    Please decide which of these are relevant web links for a brochure about the company, 
    respond with the full https URL in JSON format.
    Do not include Terms of Service, Privacy, email links.

    Links (some might be relative links):

    """
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt
    

Send request to open ai model

In [ ]:
def response_api(url):
  response = client.chat.completions.create(
          model="openai/gpt-oss-20b:free",
          messages=[
                {
                  "role": "system",
                  "content": system_one_shot_prompt
                },
                {
                    "role": "user",
                    "content": user_prompts(url)
                }
              ],

      )


  return response.choices[0].message.content


In [ ]:
response_api("https://github.com/Jessica-ovabor")

'{\n    "links": [\n        {"type": "about page", "url": "https://github.com/Jessica-ovabor"},\n        {"type": "portfolio page", "url": "https://jessica-ovabor-portfolio.vercel.app/"},\n        {"type": "linkedin page", "url": "https://www.linkedin.com/in/jovabor/"},\n        {"type": "twitter page", "url": "https://x.com/jovaboh"},\n        {"type": "medium page", "url": "https://medium.com/@jessicaovabor"},\n        {"type": "orcid page", "url": "https://orcid.org/0009-0004-3558-6897"}\n    ]\n}'